<a href="https://colab.research.google.com/github/nxxk23/AI-Engineer/blob/main/extract/gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install easyocr gradio pythainlp
!sudo apt-get install ghostscript

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os

resume_directory = '/content/drive/MyDrive/AIEngineer/resume/resume_LLM'
pdf_files = [os.path.join(resume_directory, f) for f in os.listdir(resume_directory) if f.endswith('.pdf')]

# Print the list of PDF files to verify
print(pdf_files)

['/content/drive/MyDrive/AIEngineer/resume/resume_LLM/Resume BA 2.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/Resume BA 3.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/Resume ba 5.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/Resume ba2.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/Resume baa.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/resume 6.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/resume 7.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/resume 2.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/resume 3.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/resume 4.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/resume 5.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/resume 1.pdf']


## gradio

In [20]:
import os
import re
import numpy as np
import pandas as pd
import easyocr
import subprocess
from pythainlp.phayathaibert.core import NamedEntityTagger
import gradio as gr
from PIL import Image

# Function to convert PDF to images using Ghostscript
def convert_pdf_to_images(pdf_file, output_folder="/content/images", dpi=300):
    try:
        # Ensure the output directory exists
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

        # Command to call Ghostscript
        output_format = os.path.join(output_folder, "page_%03d.png")
        gs_command = [
            "gs",
            "-sDEVICE=png16m",   # Output as PNG images
            f"-r{dpi}",           # Set the resolution (DPI)
            "-o", output_format,  # Output format
            pdf_file              # Input PDF file
        ]

        subprocess.run(gs_command, check=True)

        # Collect the generated image file paths
        image_files = sorted([os.path.join(output_folder, f) for f in os.listdir(output_folder) if f.endswith(".png")])
        return image_files

    except subprocess.CalledProcessError as e:
        print(f"Ghostscript error: {e}")
        return []

# Function to extract text from a single image
def extract_text_from_image(image_path):
    reader = easyocr.Reader(['th', 'en'], gpu=True)  # Enable GPU if available for faster processing
    image = Image.open(image_path)
    image_np = np.array(image)
    result = reader.readtext(image_np)
    sorted_data = sorted(result, key=lambda x: x[0][0][1])  # Sort based on y-coordinate
    plain_text = "\n".join([text for _, text, _ in sorted_data])
    return plain_text

# Function to tag and clean the extracted text
def tag_and_clean_text(text):
    tagger = NamedEntityTagger()  # No device argument, as it uses GPU automatically if available
    unwanted_terms = [
        'ที่อยู่', 'โทรศัพท์', 'อีเมล', 'linkedin', ':', ',', '-', '|',
        'ประวัติส่วนตัว', 'เกี่ยวกับฉัน', 'about me', 'ชื่อ', 'สกุล', 'tell', 'โทร', 'โทรงาน',
        'ชื่อเล่น', 'อายุ', 'วันเกิด', 'พุทธ', 'ศาสนา', 'สัญชาติ', 'phone',
        'ช่องทางการติดต่อ', '_', 're sume', 'resume', 'resu me'
    ]
    unwanted_pattern = '|'.join(map(re.escape, unwanted_terms))

    try:
        ner = tagger.get_ner(text, tag=True)
        pattern = r'<(?!ORGANIZATION|PERCENT|TIME)[^>]+>[^<]+</[^>]+>'
        cleaned_ner = re.sub(pattern, '', ner)
        cleaned_ner = re.sub(r'</?(ORGANIZATION|PERCENT|TIME)>', '', cleaned_ner)
        cleaned_ner = re.sub(unwanted_pattern, '', cleaned_ner)
        cleaned_ner = re.sub(r'\bal\b', 'ai', cleaned_ner, flags=re.IGNORECASE)
        return ner.strip(), cleaned_ner.strip()
    except Exception as e:
        print(f"Error in tag_and_clean_text: {e}")
        return "", text

# Function to process a single PDF
def process_single_pdf(pdf_file):
    results = []
    try:
        # Convert PDF to images using Ghostscript
        images = convert_pdf_to_images(pdf_file.name)
        raw_text = ""
        for image_path in images:
            raw_text += extract_text_from_image(image_path) + "\n"

        # Tag and clean text
        tagged_text, cleaned_text = tag_and_clean_text(raw_text)
        results.append({
            "PDF File": os.path.basename(pdf_file.name),
            "Raw_Text": raw_text,
            "Tagged_Text": tagged_text,
            "Cleaned_Text": cleaned_text
        })

    except Exception as e:
        print(f"Error in process_single_pdf: {e}")

    # Convert results to a DataFrame
    df = pd.DataFrame(results)
    return df

# Function to process multiple PDFs
def process_multiple_pdfs(pdf_files):
    all_results = pd.DataFrame()
    for pdf_file in pdf_files:
        df = process_single_pdf(pdf_file)
        all_results = pd.concat([all_results, df], ignore_index=True)
    return all_results

# Save DataFrame to CSV
def save_dataframe(df):
    try:
        csv_path = '/content/output.csv'
        df.to_csv(csv_path, index=False, encoding='utf-8-sig')
        return csv_path
    except Exception as e:
        print(f"Error saving DataFrame: {e}")
        return ""

# Gradio Interface Function
def gradio_interface(pdf_files):
    try:
        df = process_multiple_pdfs(pdf_files)
        csv_path = save_dataframe(df)
        return df, csv_path
    except Exception as e:
        print(f"Error in gradio_interface: {e}")
        return pd.DataFrame(), ""

# Gradio UI
iface = gr.Interface(
    fn=gradio_interface,
    inputs=[gr.Files(label="Upload PDF Files")],  # Allow multiple files
    outputs=[
        gr.Dataframe(label="Extracted and Cleaned Data", height=500, min_width=800),
        gr.File(label="Download CSV")  # Add a file download button
    ],
    title="PDF to Text and Data Cleaner"
)

# Launch the interface
iface.launch(share=True, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://10ac405f8a5bdf718f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/easyocr/detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(copyStateDict(torch.load(trained_model, ma

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://10ac405f8a5bdf718f.gradio.live


In [21]:
import pandas as pd
check = pd.read_csv('/content/output.csv')
check

,PDF File,Raw_Text,Tagged_Text,Cleaned_Text
0,Resume BA 2.pdf,"พิมพ์นารา วัฒนชัย\nที่อยู่: 789 ถนนรัชดาภิเษก,...",<PERSON>พ</PERSON><PERSON>พิมพ์</PERSON><PERSO...,st\nเป้าหมายในการทำงาน\nนำความเชี่ยวชาญด้านการ...
1,Resume ba 5.pdf,081-456-7890\nนัทธมน ฺรัตนโชติ\n nattamon. rad...,<PHONE>08</PHONE><PERSON>1-4</PERSON><PERSON>5...,เป้าหมายในการ\n มุ่งมันในการนำทักษะด้านการจัดก...
2,resume 6.pdf,linkedin:\nนภัสสร วิวัฒนาวงศ์\nlinkedin.com i...,linkedin:<PERSON>\n</PERSON><PERSON>น</PERSON>...,in\n\n วิศวกรปัญญาประดิษฐ์\n ประสบการณ์การทำงา...
3,Resume BA 3.pdf,จิตรลดา\n086-234-5678\njitlada.s@domain. com\...,<PERSON> </PERSON><PERSON>จิตร</PERSON><PERSON...,เสริม\n business analyst\nเป้าหมายในการทำงาน\n...
4,Resume ba2.pdf,ศุภกิจฺ มงคลชัย\n089-234-5678\n supakit .m@do...,<PERSON> </PERSON><PERSON>ศุภ</PERSON><PERSON>...,ประสบการณ์การ\nเป้าหมายในการทำงาน\nflnancial a...
5,Resume baa.pdf,"ธนกร อินทรีย์พงษ์\n ที่อยู่:\n456 ถนนพระราม 3,...",<PERSON>ธ</PERSON><PERSON>ธน</PERSON><PERSON>ก...,เป้าหมายในการทำงาน\nมุ่งมันที่จะใช้ความรู้และท...
6,resume 2.pdf,วงศ์เจริญ\nณ็ชพล\n วิศวกรปัญญาประดิษฐ์\n ประว...,<PERSON> วงศ์เจริญ\nณ็ชพล</PERSON>\n วิศวกรปัญ...,วิศวกรปัญญาประดิษฐ์\n ประวัติการศึกษา\nมหาวิทย...
7,resume 3.pdf,re sume\nประวัติส่วนตัว\n090-123-4567\n pimcha...,re sume\nประวัติส่วนตัว<PHONE>\n09</PHONE><PHO...,ผู้ เชี่ยวชาญด้าน ai ที่มีประสบการณ์ 4 เขี\nใน...
8,resume 4.pdf,อาทิตย์\nรัตนวิจิตร\n วิศวกรปัญญาประดิษฐ์\nปร...,<PERSON> </PERSON><PERSON>อาทิตย์\nรัตนวิจิตร<...,วิศวกรปัญญาประดิษฐ์\nประสบการณํ ทำงาน\nวิศวกร ...
9,resume 7.pdf,ปัญญา วิริยะชัย\n วิศวกรปัญญาประดิษฐ์\nประสบกา...,<PERSON>ป</PERSON><PERSON>ปัญญา วิริยะชัย</PER...,วิศวกรปัญญาประดิษฐ์\nประสบการณ์การฝึกงาน\nบัณฑ...


## code

In [17]:
import os
import easyocr
import pandas as pd
import shutil
from pythainlp.phayathaibert.core import NamedEntityTagger
import re
from tempfile import TemporaryDirectory

# Define the function to extract text from images
def extract_text_from_image(image_path):
    reader = easyocr.Reader(['th', 'en'])
    result = reader.readtext(image_path)
    sorted_data = sorted(result, key=lambda x: x[0][0][1])
    plain_text = "\n".join([text for _, text, _ in sorted_data])
    return plain_text

def convert_pdf_to_images(pdf_path, output_dir):
    pdf_filename = os.path.splitext(os.path.basename(pdf_path))[0]
    output_pattern = os.path.join(output_dir, f"{pdf_filename}_%d.png")
    os.system(f'gs -dBATCH -dNOPAUSE -sDEVICE=pngalpha -r300 -sOutputFile="{output_pattern}" "{pdf_path}"')

# Define the function to process and clean multiple PDFs
def process_pdfs_in_directory(pdf_dir):
    tagger = NamedEntityTagger()
    unwanted_terms = [
        'ที่อยู่', 'โทรศัพท์', 'อีเมล', 'linkedin', ':', ',', '-', '|',
        'ประวัติส่วนตัว', 'เกี่ยวกับฉัน', 'about me', 'ชื่อ', 'สกุล', 'tell', 'โทร', 'โทรงาน',
        'ชื่อเล่น', 'อายุ', 'วันเกิด', 'พุทธ', 'ศาสนา', 'สัญชาติ', 'phone',
        'ช่องทางการติดต่อ', '_', 're sume', 'resume', 'resu me'
    ]
    # If some terms might start with special characters, re.escape will handle them
    unwanted_pattern = '|'.join(map(re.escape, unwanted_terms))

    def tag_and_clean_text(text):
        ner = tagger.get_ner(text, tag=True)
        pattern = r'<(?!ORGANIZATION|PERCENT|TIME)[^>]+>[^<]+[^>]+>'
        cleaned_ner = re.sub(pattern, '', ner)
        cleaned_ner = re.sub(r'(<ORGANIZATION>|<PERCENT>|<TIME>)', '', cleaned_ner)
        cleaned_ner = re.sub(unwanted_pattern, '', cleaned_ner)
        cleaned_ner = re.sub(r'\bal\b', 'ai', cleaned_ner, flags=re.IGNORECASE)
        return ner.strip(), cleaned_ner.strip()

    results = []
    for pdf_file in os.listdir(pdf_dir):
        if pdf_file.lower().endswith('.pdf'):
            pdf_path = os.path.join(pdf_dir, pdf_file)

            with TemporaryDirectory() as temp_dir:
                images_dir = os.path.join(temp_dir, 'images')
                os.makedirs(images_dir, exist_ok=True)

                # Convert the PDF to images
                convert_pdf_to_images(pdf_path, images_dir)

                # Extract text from all images related to the PDF file
                raw_text = ""
                pdf_filename = os.path.splitext(os.path.basename(pdf_path))[0]
                for img_file in os.listdir(images_dir):
                    if img_file.startswith(pdf_filename):
                        img_path = os.path.join(images_dir, img_file)
                        raw_text += extract_text_from_image(img_path) + "\n"

                # Tag and clean the extracted text
                tagged_text, cleaned_text = tag_and_clean_text(raw_text)

                # Append the result to the list
                results.append({
                    "PDF File": pdf_file,
                    "Raw_Text": raw_text,
                    "Tagged_Text": tagged_text,
                    "Cleaned_Text": cleaned_text
                })

    # Create a DataFrame with all results
    df = pd.DataFrame(results)
    return df


In [18]:
pdf_dir = '/content/drive/MyDrive/AIEngineer/resume/resume_LLM'
df = process_pdfs_in_directory(pdf_dir)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Hardwa

In [19]:
df

,PDF File,Raw_Text,Tagged_Text,Cleaned_Text
0,Resume BA 2.pdf,พิมพ์นารา วัฒนชัย\nที่อยู่:\n789 ถนนรัชดาภิเษก...,<PERSON>พ</PERSON><PERSON>พิมพ์</PERSON><PERSO...,เป้าหมายในการทำงาน\nนำความเชี่ยวชาญด้านการวิเค...
1,Resume BA 3.pdf,จิตรลดา\n086-234-5678\n jitlada.s@domain.com\...,<PERSON> </PERSON><PERSON>จิตร</PERSON><PERSON...,เสริม\n business บริษัท modern solutions\nbusi...
2,Resume ba 5.pdf,081-456-7890\nนัทธมน ฺรัตนโชติ\n nattamon. rad...,<PHONE>08</PHONE><PERSON>1-4</PERSON><PERSON>5...,เป้าหมายในการ\n มุ่งมันในการนำทักษะด้านการจัดก...
3,Resume ba2.pdf,ศุภกิจฺ มงคลชัย\n089-234-5678\n supakit.m@dom...,<PERSON> </PERSON><PERSON>ศุภ</PERSON><PERSON>...,ประสบการณ์การ\nเป้าหมายในการทำงาน\nfinancialt ...
4,Resume baa.pdf,"ธนกร อินทรีย์พงษ์\n ที่อยู่:\n456 ถนนพระราม 3,...",<PERSON>ธ</PERSON><PERSON>ธน</PERSON><PERSON>ก...,เป้าหมายในการทำงาน\nมุ่งมันที่จะใช้ความรู้และท...
5,resume 6.pdf,linkedin:\nนภัสสร วิวัฒนาวงศ์\nlinkedin.com i...,linkedin:<PERSON>\n</PERSON><PERSON>น</PERSON>...,in\n\n วิศวกรปัญญาประดิษฐ์\n\n ประสบการณ์การทำ...
6,resume 7.pdf,ปัญญา วิริยะชัย\n วิศวกรปัญญาประดิษฐ์\nประสบกา...,<PERSON>ป</PERSON><PERSON>ปัญญา วิริยะชัย</PER...,วิศวกรปัญญาประดิษฐ์\nประสบการณ์การฝึกงาน\nบัณฑ...
7,resume 2.pdf,วงศ์เจริญ\nณ็ชพล\n วิศวกรปัญญาประดิษฐ์\n ประว...,<PERSON> วงศ์เจริญ\nณ็ชพล</PERSON>\n วิศวกรปัญ...,วิศวกรปัญญาประดิษฐ์\n ประวัติการศึกษา\n</ORGAN...
8,resume 3.pdf,re sume\n ประวัติส่วนตัว\n090-123-4567\npimcha...,re sume\n ประวัติส่วนตัว<PHONE>\n09</PHONE><PH...,ผู้ เชี่ยวชาญด้าน ai ที่มีประสบการณ์ 4 เขี\nใน...
9,resume 4.pdf,อาทิตย์\nรัตนวิจิตร\n วิศวกรปัญญาประดิษฐ์\nปร...,<PERSON> </PERSON><PERSON>อาทิตย์\nรัตนวิจิตร<...,วิศวกรปัญญาประดิษฐ์\nประสบการณํ ทำงาน\nวิศวกร ...


> `tag ต่างกัน ส่งผลให้ cleaned_text ต่างกัน`

In [27]:
print(check['Tagged_Text'].iloc[0])

<PERSON>พ</PERSON><PERSON>พิมพ์</PERSON><PERSON>นารา วัฒนชัย</PERSON>
ที่อยู่:<LOCATION> </LOCATION><LOCATION>789</LOCATION><LOCATION> </LOCATION><LOCATION>ถนน</LOCATION><LOCATION>รัชดาภิเษก</LOCATION>,<LOCATION> </LOCATION><LOCATION>กรุงเทพฯ</LOCATION><LOCATION>103</LOCATION><ZIP>10</ZIP>
โทรศัพท์:<PHONE> 08</PHONE><PHONE>9</PHONE><PHONE>-</PHONE><PHONE>765-4321</PHONE>
อีเมล:<EMAIL> pi</EMAIL><PERSON>mn</PERSON><PERSON>ara</PERSON><PERSON>.</PERSON><EMAIL>w@domain.com</EMAIL>
 linkedin:<EMAIL> linkedin</EMAIL><URL> .com in pimnara w
busiess anal v</URL>st
เป้าหมายในการทำงาน
นำความเชี่ยวชาญด้านการวิเคราะห์และการจัดการโครงการมาช่วยองค์กรปรับปรุงกระบวนการ
 ทำงานและบรรลุเป้าหมายธุรกิจอย่างมีประสิทธิภาพ
ทักษะ
การวิเคราะห์เชิงกลยุทธ์: swot, pestel
การจัดการโครงการ: microsoft project., asana
การสื่อสาร: การนำเสนอ,การจัดทำรายงาน
การจัดการข้อมูล: excel., sql, power bi
การศึกษา
 ปริญญาโท สาขา<ORGANIZATION>วิทยา</ORGANIZATION>การจัดการ
2015
2017<LOCATION>
</LOCATION><ORGANIZATION>มหาวิทยาลั</OR

In [28]:
print(df['Tagged_Text'].iloc[0])

<PERSON>พ</PERSON><PERSON>พิมพ์</PERSON><PERSON>นารา วัฒนชัย</PERSON>
ที่อยู่:<LOCATION>
</LOCATION><LOCATION>789</LOCATION><LOCATION> </LOCATION><LOCATION>ถนน</LOCATION><LOCATION>รัชดาภิเษก</LOCATION>,<LOCATION> </LOCATION><LOCATION>กรุงเทพฯ</LOCATION><ZIP> 103</ZIP><ZIP>10</ZIP>
โทรศัพท์:<PHONE> 08</PHONE><PHONE>9</PHONE><PHONE>-</PHONE><PHONE>765-4321</PHONE>
อีเมล:<EMAIL> pi</EMAIL><PERSON>mn</PERSON><EMAIL>ara</EMAIL><EMAIL>.</EMAIL><EMAIL>w@domain. com</EMAIL><URL>
linkedin</URL>:<URL> linkedin . com in</URL><EMAIL> pi</EMAIL><URL>mnara w
busiess analvst</URL>
เป้าหมายในการทำงาน
นำความเชี่ยวชาญด้านการวิเคราะห์และการจัดการโครงการมาช่วยองค์กรปรับปรุงกระบวนการ
ทำงานและบรรลุเป้าหมายธุรกิจอย่างมีประสิทธิภาพ
ทักษะ
การวิเคราะห์เชิงกลยุทธ์: swot pestel
การจัดการโครงการ: microsoft project, asana
การสื่อสาร: การนำเสนอ,การจัดทำรายงาน
การจัดการข้อมูล: excel., sql, power bi
การศึกษา
 ปริญญาโท สาขา<ORGANIZATION>วิทยา</ORGANIZATION>การจัดการ
2015
2017<ORGANIZATION>
</ORGANIZATION><ORGANIZATION>

In [37]:
print(check['Cleaned_Text'].iloc[0],"\n\n", "----------", "\n",df['Cleaned_Text'].iloc[0])

st
เป้าหมายในการทำงาน
นำความเชี่ยวชาญด้านการวิเคราะห์และการจัดการโครงการมาช่วยองค์กรปรับปรุงกระบวนการ
 ทำงานและบรรลุเป้าหมายธุรกิจอย่างมีประสิทธิภาพ
ทักษะ
การวิเคราะห์เชิงกลยุทธ์ swot pestel
การจัดการโครงการ microsoft project. asana
การสื่อสาร การนำเสนอการจัดทำรายงาน
การจัดการข้อมูล excel. sql power bi
การศึกษา
 ปริญญาโท สาขาวิทยาการจัดการ
2015
2017มหาวิทยาลัยเกษตรศาสตร์
ประสบการณ์การทำงาน
senior business analyst บริษัท strategic partners
ทำงานร่วมกับทีมผู้บริหารเพื่อระบุความต้องการและโอกาสในการปรับปรุงกระบวนการธุรกิจ
พัฒนา roadmap สำหรับการเปลี่ยนแปลงและปรับปรุงระบบเทคโนโลยีสารสนเทศ
 วิเคราะห์และนำเสนอข้อมูลสำคัญที่มีผลต่อการตัดสินใจเชิงกลยุทธ์
business analyst บริษัท delta innovations 

 ---------- 
 เป้าหมายในการทำงาน
นำความเชี่ยวชาญด้านการวิเคราะห์และการจัดการโครงการมาช่วยองค์กรปรับปรุงกระบวนการ
ทำงานและบรรลุเป้าหมายธุรกิจอย่างมีประสิทธิภาพ
ทักษะ
การวิเคราะห์เชิงกลยุทธ์ swot pestel
การจัดการโครงการ microsoft project asana
การสื่อสาร การนำเสนอการจัดทำรายงาน
การจัดการข้อมูล excel. sq